In [3]:
import sys


In [106]:
sys.argv.pop()

'./student_export.text'

In [24]:
sys.argv

['/Users/aciuffo/Virtualenvs/python2/lib/python2.7/site-packages/ipykernel/__main__.py']

In [108]:
sys.argv.append('./student_export.text')

In [97]:
import sys
import re
import datetime
import unicodedata
import csv

# I kind of hate this
#from __future__ import print_function

# this may be unecessary - double check
import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage 
from apiclient import errors 

SCOPES = 'https://www.googleapis.com/auth/drive'
#SCOPES = 'https://www.googleapis.com/auth/drive.file' # try this one!
# SCOPES = 'https://www.googleapis.com/auth/drive.metadata.readonly'
CLIENT_SECRET_FILE = 'client_secret_qs.json'
APPLICATION_NAME = 'Drive API Python Quickstart'

In [101]:
errors.HttpError

googleapiclient.errors.HttpError

In [10]:
def strip_accents(s):
    s = unicode(s, "utf-8")
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


In [11]:
def get_valid_filename(s):
    """
    Return the given string converted to a string that can be used for a clean
    filename. Remove leading and trailing spaces
    """
    s = s.strip()
    s = strip_accents(s)
    return re.sub(r'(?u)[^-\w., ]', '', s)

In [12]:
def fileRead(fname):
    '''
    read a file into a list, strip out all accented and special characters, leading spaces
    '''
    lines = []
    try:
        with open(fname) as f:
            for each in f:
                each = get_valid_filename(each)
                lines.append(each.strip('\n'))
            return(lines)
    except (OSError, IOError) as e:
        print 'error reading file:', fname, e
        return(False)

In [13]:
def pathify(parts = [], basepath = ''):
    '''
    create a path from a list of strings
    
    accepts:
        * parts (list): list of strings ['part_one', 'part_two', 'part_three']
        * basepath (string): string to append to the start of the path (default: ./)
    
    returns:
        * path: basePath/part_one/part_two/part_three/
    '''
    path = basepath
    if len(parts ) > 0:
        for each in parts:
            if re.match('.*\/$', each):
                path = path + each
            else:
                path = path + each + '/'
    return(path)

In [14]:
def getCredentials():
    home_dir = os.path.expanduser('~')
    credential_dir = os.path.join(home_dir, '.credentials')
    if not os.path.exists(credential_dir):
        try:
            os.makedirs(credential_dir)
        except (IOError, OSError) as e:
            print(e)
        

    credential_path = os.path.join(credential_dir, 'drive-python.json')

    store = Storage(credential_path)
    credentials = store.get()
    if not credentials or credentials.invalid:
        flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)
        flow.user_agent = 'foo'
        credentials = tools.run_flow(flow, store, None)
    return(credentials)

In [15]:
class GDriveError(Exception):
    pass

In [120]:
class gDrive():
    '''
    creates a google drive interface object
    
    Accepts:
    google drive v3 service object: (discover.build('drive', 'v3', credentials = credentials_object)
    
    '''
    def __init__(self, object):
        if  not isinstance(object, discovery.Resource):
            print ('Error: googleapicleint.discovery.Resource object expected')
            print ('{:>5}create a resource object:'.format(''))
            print ('{:>10}credentials = getCredentials(credJSON = "cleint_secret.json")'.format(''))
            print ('{:>10}service = discovery.build("drive", "v3", credentials=credentials)'.format(''))
            print ('{:>10}myDrive = gDrive(service)'.format(''))
            return(None)
        self.service = object
        # https://developers.google.com/drive/v3/web/mime-types
        self.mimeTypes = {'audio': 'application/vnd.google-apps.audio',
                          'docs': 'application/vnd.google-apps.document',
                          'drawing': 'application/vnd.google-apps.drawing',
                          'file': 'application/vnd.google-apps.file',
                          'folder': 'application/vnd.google-apps.folder',
                          'forms': 'application/vnd.google-apps.form',
                          'mymaps': 'application/vnd.google-apps.map',
                          'photos': 'application/vnd.google-apps.photo',
                          'slides': 'application/vnd.google-apps.presentation',
                          'scripts': 'application/vnd.google-apps.script',
                          'sites': 'application/vnd.google-apps.sites',
                          'sheets': 'application/vnd.google-apps.spreadsheet',
                          'video': 'application/vnd.google-apps.video'}
        
        # fields to include in partial responses
        # https://developers.google.com/apis-explorer/#p/drive/v3/drive.files.create
        self.fields = ['id', 'parents', 'mimeType', 'webViewLink', 'size', 'createdTime', 'trashed']
    
#     types = property()
    
    @property
    def types(self):
        '''
        Display supported mimeTypes
        '''
        print('supported mime types:')
        for key in self.mimeTypes:
            #print('%10s: %s' % (key, self.mimeTypes[key]))
            print('{:8} {val}'.format(key+':', val=self.mimeTypes[key]))
    
#     def quote(self, string):
#         '''
#         add double quotes arounda string
#         '''
#         return('"'+str(string)+'"')
    

    
    def add(self, name = None, mimeType = False, parents = None, 
            fields = 'webViewLink, mimeType, id', sanitize = True):
        '''
        add a file to google drive:
        
        args:
            name (string): human readable name
            mimeType (string): mimeType (see self.mimeTypes for a complete list)
            parents (list): list of parent folders
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 
                'size', 'createdTime', 'trashed'
                'id'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
            
        '''

        fieldsExpected = self.fields
        fieldsProcessed = []
        fieldsUnknown = []
        
        if sanitize:
            # remove whitespace and unknown options
            for each in fields.replace(' ','').split(','):
                if each in fieldsExpected:
                    fieldsProcessed.append(each)
                else:
                    fieldsUnknown.append(each)
        else:
            fieldsProcessed = fields.split(',')
            
        if len(fieldsUnknown) > 0:
            print ('unrecognized fields: {}'.format(fieldsUnknown))
        
        
        body={}
        if name is None:
            print('specify a folder or file name')
            return(False)
        else:
            body['name'] = name
        
        if mimeType in self.mimeTypes:
            body['mimeType'] = self.mimeTypes[mimeType]
        
        if isinstance(parents, list):
            body['parents'] = parents
        elif parents:
            body['parents'] = [parents]
        
        try:
            result = self.service.files().create(body=body, fields=','.join(fieldsProcessed)).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        
        
        #body = {'name':'release the schmoo!', 'mimeType':'application/vnd.google-apps.folder', 'parents':["0BzC-V2QIsGRGWXNxNmhjc0FITDQ"]}
# service.files().create(body=body).execute()
        

    
    def search(self, name = None, trashed = None, mimeType = False, fuzzy = False, date = None, dopperator = '>', 
               parents = None, orderBy = 'createdTime', quiet = True):
        '''
        search for an item by name and other properties in google drive
        
        args:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parents = (string): google drive file id string
            orderBy = (comma separated string): order results assending by keys below - default createdTime:
                        'createdTime', 'folder', 'modifiedByMeTime', 
                        'modifiedTime', 'name', 'quotaBytesUsed', 
                        'recency', 'sharedWithMeTime', 'starred', 
                        'viewedByMeTime'
                        
            
        returns:
            list of file dict
        '''
        features = ['name', 'trashed', 'mimeType', 'date', 'parents']
        build = {'name' : 'name {} "{}"'.format(('contains' if fuzzy else '='), name),
                 'trashed' : 'trashed={}'. format(trashed),
                 'mimeType' : 'mimeType="{}"'.format(self.mimeTypes[mimeType] if mimeType in self.mimeTypes else ''),
                 'parents': '"{}" in parents'.format(parents),
                 'date': 'modifiedTime{}"{}"'.format(dopperator, date)}

        # provides for setting trashed to True/False if the input is not None
        if not isinstance(trashed, type(None)):
            # set to true as the variable is now in use, but it's value has been set above
            trashed = True
        
        qList = []

        # evaluate feature options; if they are != None/False, use them in building query
        for each in features:
            if eval(each):
                qList.append(build[each])
                
        if not quiet:
            print(' and '.join(qList))
                
        try:
            # build a query with "and" statements
            result = self.service.files().list(q=' and '.join(qList), orderBy=orderBy).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)

    def ls(self, *args, **kwargs):
        '''
        List files in google drive using any of the following properties:
            
        accepts:
            name (string): item name in google drive - required
            trashed (bool): item is not in trash - default None (not used)
            mimeType = (string): item is one of the known mime types (gdrive.mimeTypes) - default None
            fuzzy = (bool): substring search of names in drive
            date = (RFC3339 date string): modification time date string (YYYY-MM-DD)
            dopperator (date comparison opprator string): <, >, =, >=, <=  - default >
            parent = (string): google drive file id string    
        '''
        try:
            result = self.search(*args, **kwargs)
            for eachFile in result.get('files', []):
                print('name: {f[name]}, ID:{f[id]}, mimeType:{f[mimeType]}'.format(f=eachFile))
            return(result)
        except GDriveError as e:
            raise GDriveError(e)
            
    
    
    def getprops(self, fileId = None, fields = 'parents, mimeType, webViewLink', sanitize=True):
        '''
        get a file or folder's properties based on google drive fileId
        
        for a more complete list: https://developers.google.com/drive/v3/web/migration
        
        args:
            fileId (string): google drive file ID
            fields (comma separated string): properties to query and return any of the following:
                'parents', 'mimeType', 'webViewLink', 'size', 'createdTime', 'trashed'
            sanitize (bool): remove any field options that are not in the above list - false to allow anything
            
        returns:
            list of dictionary - google drive file properties
            
        raises GDriveError
        '''
        fieldsExpected = self.fields
        
        fieldsProcessed = []
        fieldsUnknown = []

        if sanitize:
            # remove whitespace and unknown options
            for each in fields.replace(' ','').split(','):
                if each in fieldsExpected:
                    fieldsProcessed.append(each)
                else:
                    fieldsUnknown.append(each)
        else:
            fieldsProcessed = fields.split(',')
        if len(fieldsUnknown) > 0:
            print ('unrecognized fields: {}'.format(fieldsUnknown))
        
        try:
            result = self.service.files().get(fileId=fileId, fields=','.join(fieldsProcessed)).execute()
            return(result)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
        

                
    
    
    def parents(self, fileId):
        """get a file's parents.

        Args:
            fileId: ID of the file to print parents for.
        
        raises gDError
        """
        try:
            parents = self.service.files().get(fileId=fileId, fields='parents').execute()
            return(parents)
        except errors.HttpError as e:
            raise GDriveError(e)
            return(False)
    
    def rm(self):
        pass

In [115]:
def main():
    
    BASEFOLDER = '0B9WTleJ1MzaYcmdmTWNNNF9pa1E'
    
    credentials = getCredentials()
    # http = credentials.authorize(httplib2.Http())
    try:
        service = discovery.build('drive', 'v3', credentials=credentials)
    except Exception as e:
        print('Error communicating with Google: {}'.format(e))
        print('exiting')
        return(False)

    myDrive = gDrive(service)

    # path to desktop of this user (this should work on macs)
    desktopPath = pathify([os.path.expanduser('~')], 'Desktop')

    # Fields expected in CSV file containing student data
    # move this into a resource file? - see the folder list below
    expected = ['ClassOf', 'LastFirst', 'Student_Number']

    # map of CSV fields to elements in list
    headerMap = {}

    # list of folders to populate into each students' folder
    if os.path.exists(desktopPath+'folders.txt'):
        fileList = desktopPath + 'folders.txt'
    else:
        fileList = './folders.txt'

    # list of student information
    studentInfo = ''
    

    # list containing CSV
    studentCSV = []

    # list of folders to add under each students' path
    folders = fileRead(fileList)

    try:
        print('input file: {}'.format(sys.argv[1]))
        studentInfo = sys.argv[1]
    except IndexError as e:
        print('Error: please specify a CSV file containing student information')

    if folders:
        print('Folders to be created for each stuent: {}'.format(len(folders)))
        print('{:>5}List of folders to be created:'.format(''))
        for folder in folders:
            print('{:>10}{}'.format('', folder))
    else:
        print('please place a file named "folders.txt" containing a list of folders (one on each line)on the Desktop')
        print('exiting')
        return(False)

    # read the csv file in Universal newline mode (rU)
    try:
        with open(studentInfo, 'rU') as csvfile:
            csvreader = csv.reader(csvfile)
            for row in csvreader:
                studentCSV.append(row)
    except (OSError, IOError) as e:
        print 'error reading file:', studentInfo, e
        print 'exiting'
        return(False)

    # check if there is content in the CSV
    if len(studentCSV) > 1:
        print('{} student records found in {}'.format(len(studentCSV)-1, studentInfo))
    else:
        print 'no student records found in {}'.format(studentInfo)
        return(False)

    # check for expected headers in CSV data
    # pop the headers from the list
    csvHeader = studentCSV.pop(0)
    missingHeaders = []
    for each in expected:
        if each not in csvHeader:
            missingHeaders.append(each)

    if len(missingHeaders) > 0:
        print 'ERROR'
        for each in missingHeaders:
            print studentInfo, ' missing header: ', each
        print 'please recreate file', studentInfo, 'with all required headers:'
        print '     ', expected
        print 'stopping'
        return(False)

    # map headers to their index
    for index, value in enumerate(csvHeader):
        headerMap[value]=index
    
    # process the CSV and start creating folders
    
    # Create a dictionary of recently used google drive (gd) folders
    gdRecentFolders = {}
    
    # create a list of failures
    studentFailures = []
# need to deal with errors here - each myDrive method needs one...

    for index, student in enumerate(studentCSV):
        studentClassFolder = 'Class Of '+student[headerMap['ClassOf']]
        # check if the folder has been recently accessed and recorded in the gdRecentFolders dict
        if studentClassFolder in gdRecentFolders:
            # reuse the folder information from previous access
            gdParentFolder = gdRecentFolders[studentClassFolder]
            
        # else search for the folder on google drive
        else:
            # search for matching folders and order from oldest to newest
            try:
                gdFolderSearch = myDrive.search(name = studentClassFolder, parents = BASEFOLDER, 
                                               orderBy = 'createdTime', trashed = False, mimeType = 'folder')
            except GDriveError as e:
                print('failed to process record: {}'.format(student))
                print(e)
                studentFailures.append(student)
                continue
            
            if len(gdFolderSearch['files']) < 1:
                print('failed to locate destination folder {}'.format(BASEFOLDER))
                print('please update the URL for the folder that will hold portfolios')
                print('exiting')
                return(False)
            
            # if no results are returned, create a folder
            if len(gdFolderSearch['files'])==0:
                try:
                    gdParentFolder = myDrive.add(name=studentClassFolder, 
                                                 mimeType = 'folder', parents=BASEFOLDER)
                except GDriveError as e:
                    print('failed to process record {}'.format(student))
                    print(e)
                    studentFailures.append(student)
                    continue
                    
                except Exception as e:
                    print('Unexected error: {}'.format(e))
                    print('exiting')
                    print('please share the error above with the developer')
                    return(False)
            # use the oldest existing folder ignoring all others
            else:
                gdParentFolder = gdFolderSearch['files'][0]

            # raise a red flag if there are 2 or more matching folders
            if len(gdFolderSearch['files']) > 1:
                print('Warning: {} folders for {} found. Consider consolidating these into one single folder.'
                      .format(len(gdFolderSearch['files']), studentClassFolder))
                print('Using the following folder: {}'
                      .format(myDrive.getprops(fileId=gdParentFolder.get('id')).get('webViewLink'))) 
            
            # record the recently used folder
            gdRecentFolders[studentClassFolder] = gdParentFolder
            
            # process students

            
    
#     if len(studentFailures) > 0:
#         print('student failures')
#         print(studentFailures)

In [116]:
foo = main()


input file: ./student_export.text
Folders to be created for each stuent: 5
     List of folders to be created:
          08-Grade
          09-Grade
          10-Grade
          11-Grade
          12-Grade
29 student records found in ./student_export.text


In [81]:
help(testDrive.add)

Help on method add in module __main__:

add(self, name=None, mimeType=False, parents=None) method of __main__.gDrive instance
    add a file to google drive:
    
    args:
        name (string): human readable name
        mimeType (string): mimeType (see self.mimeTypes for a complete list)
        parents (list): list of parent folders



In [ ]:
testDrive.getprops(fileId="0B9WTleJ1MzaYc2RGUkVFSktURmc").get('webViewLink')

In [121]:
credentials = getCredentials()
# http = credentials.authorize(httplib2.Http())
service = discovery.build('drive', 'v3', credentials=credentials)


testDrive = gDrive(service)


In [122]:
testDrive.add(name="release the schmoo!")

{u'id': u'0B9WTleJ1MzaYeWxVRGJKOWR0dE0',
 u'mimeType': u'application/octet-stream',
 u'webViewLink': u'https://drive.google.com/a/ash.nl/file/d/0B9WTleJ1MzaYeWxVRGJKOWR0dE0/view?usp=drivesdk'}

In [ ]:
help(testDrive.search)

In [ ]:
help(testDrive.addFile)
testDrive.mimeTypes



In [ ]:
testDrive.getprops(fileId='0B9WTleJ1MzaYcVB4aUxURENldHM_')

In [ ]:
testDrive.service.files().list(q='name = "ClassOf 2025" and "0B9WTleJ1MzaYcVB4aUxURENldHM" in parents and trashed=False', orderBy='modifiedTime').execute()


In [ ]:
result = testDrive.search(name='ClassOf 2025', fuzzy = True, parent='0B9WTleJ1MzaYcVB4aUxURENldHM', orderBy='createdTime')
if len(result['files']) > 0:
    print result['files'][0].get('id')

In [ ]:
testDrive.service.files().list(q="0B9WTleJ1MzaYcVB4aUxURENldHM").execute()

In [ ]:
testDrive.service.files().list(fields="files(id, name, webViewLink)", q="name contains 'schmoo'").execute()

In [ ]:
testDrive.service.files().get(fileId="0B9WTleJ1MzaYcVB4aUxURENldHM", fields="parents,mimeType,webViewLink, size, createdTime, trashed").execute()

In [ ]:
result = testDrive.search(name = "ClassOf", parent = "0B9WTleJ1MzaYcVB4aUxURENldHM", trashed = False)
print result

In [ ]:
testDrive.search('Portfolios APIV3', fuzzy=True, trashed = False)

In [ ]:
testDrive.addFile('schmoo', mimeType = 'folder')

In [ ]:
service.files().get(fileId="0BzC-V2QIsGRGT00xSjJHNy1Td1k", fields='parents').execute()

In [ ]:
myDrive.parents('0BzC-V2QIsGRGT00xSjJHNy1Td1k')

In [ ]:
myDrive.ls(name = 'schmoo', fuzzy = True, trashed = False)

In [ ]:
response = myDrive.search(name = 'schmoo', fuzzy = True)
print (response)

In [ ]:
myDrive.addFile('the wifes sewing', mimeType = 'folder', parents = "0BzC-V2QIsGRGWXNxNmhjc0FITDQ")

In [ ]:
myDrive.search(name = "more cows")

In [ ]:
discovery.mimetypes